In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import numpy as np
import matplotlib.pyplot as plt
import Analyzer

mk = Analyzer.MarketDB()
raw_df = mk.get_daily_price('삼성전자', '2018-05-04', '2021-06-22')

def MinMaxScaler(data):
    """ 최솟값과 최댓값을 이용하여 0 ~ 1 값으로 변환"""
    
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    
    # 0으로 나누기 에러가 발생하지 않도록 매우 작은 값(1e-7)을 더해서 나눔
    return numerator / (denominator + 1e-7)

dfx = raw_df[['open', 'high', 'low', 'volume', 'close']]
dfx = MinMaxScaler(dfx)
dfy = dfx[['close']]

x = dfx.values.tolist() # dfx 에는 종가('close')를 제외한 나머지  MinMaxScaler를 거친 데이터들이 들어있음 
y = dfy.values.tolist() # tolist() : array나 DataFrame를 list로 변환하는 함수

data_x = []
data_y = []

'''
df의 '종가'라는 칼럼에서 자신을 포함한 '5'(다섯)행 데이터를 rolling하여(?) 그 데이터를 모아 평균(mean)을 내고, 
그것을 df의 '주가이동평균선'이라는 새로운 칼럼에 담는 것입니다.

여기서 window_size를 정의하여 학습 데이터를 생성할 예정입니다.
window_size는 내가 얼마동안(기간)의 주가 데이터에 기반하여 다음날 종가를 예측할 것인가를 정하는 parameter 입니다. 
즉 내가 과거 20일을 기반으로 내일 데이터를 예측한다라고 가정했을 때는 window_size=20이 됩니다.
'''

window_size = 10 # rolling(window=정수) : '정수'행 앞의 데이터를 가져온다.
data_size = 5 

for i in range(len(y) - window_size):
    _x = x[i : i + window_size] # 다음날 종가(i + windows_size)는 포함되지 않음 10개씩 1데이터 단위로 겹치면서
    _y = y[i + window_size]     # 다음날 종가
    data_x.append(_x)
    data_y.append(_y)
print(_x, " -> ", _y)

train_size = int(len(data_y) * 0.7)
train_x = np.array(data_x[0 : train_size])
train_y = np.array(data_y[0 : train_size])

test_size = len(data_y) - train_size
test_x = np.array(data_x[train_size : len(data_x)])
test_y = np.array(data_y[train_size : len(data_y)])

# 모델 생성
model = Sequential()
model.add(LSTM(units=10, activation='relu', return_sequences=True,
input_shape=(window_size, data_size)))
model.add(Dropout(0.1))
model.add(LSTM(units=10, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

model.complie(optimizer='adam', loss='mean_squeared_error')
model.fit(train_x, train_y, epochs=60, batch_size=30)
pred_y = model.predict(test_x)

# Visualising the results
plt.figure()
plt.plot(test_y, color='red', label='real SEC stock price')
plt.plot(pred_y, color='blue', laber='predicted SEC stock price')
plt.title('SEC stock price prediction')
plt.xlabel('time')
plt.ylabel('stock price')
plt.legend()
plt.show()

print("Tomorrow's SEC price :", raw_df.close[-1] * pred_y[-1] / dfy.close[-1], "KRW")


In [2]:
raw_df

,code,date,open,high,low,close,diff,volume
0,005930,2020-04-03,47400,47600,46550,47000,200,22784682
1,005930,2020-04-06,47500,48800,47250,48700,1700,23395726
2,005930,2020-04-07,49650,50200,49000,49600,900,31524034
3,005930,2020-04-08,49600,49750,48600,48600,-1000,25010314
4,005930,2020-04-09,49750,49800,48700,49100,500,22628058
...,...,...,...,...,...,...,...,...
294,005930,2021-06-14,80800,80900,80500,80500,-500,10550078
295,005930,2021-06-15,80900,81200,80600,80900,400,10075685
296,005930,2021-06-16,81500,81900,81100,81800,900,14999855
297,005930,2021-06-17,81100,81300,80700,80900,-900,14007385
